## CNN-LSTM IoMT IDS
### CSCI 6505 Group Project
### Author: Hongwei Zhang & Koil Jat Chong
### Enhancing Intrusion Detection in Healthcare IoMT Devices Using the CNN-LSTM Model

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import warnings
warnings.simplefilter('ignore')
import sys
print("Python version:", sys.version)
print("Version info:", sys.version_info)

Python version: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
Version info: sys.version_info(major=3, minor=10, micro=11, releaselevel='final', serial=0)


In [52]:
# Load the dataset
data = pd.read_csv('./dataset/processed_data.csv')
data.head()

,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,Target
0,0.012399,0.482353,0.301176,0.000009,0.000009,0.0,0.0,0.0,0.0,0.5,...,0.465433,0.041508,9.999574e-01,0.925926,0.217273,0.466105,0.218837,1.0,1.000000,0
1,0.005670,0.417647,0.217647,0.000003,0.000003,0.0,0.0,0.1,0.0,0.5,...,0.223835,0.073505,3.288045e-10,0.333333,0.181740,0.223663,0.066195,0.9,0.153941,4
2,0.269644,1.000000,0.250980,0.000021,0.000021,0.0,0.0,0.0,0.0,0.0,...,0.187570,0.239402,3.395595e-11,0.333333,0.249725,0.187426,0.068553,0.9,0.153941,4
3,0.806794,1.000000,0.250980,0.000192,0.000192,0.0,0.0,0.0,0.0,0.0,...,0.643992,0.305027,9.996046e-01,0.925926,0.438889,0.644289,0.415955,1.0,1.000000,5
4,0.000011,0.352941,0.184314,0.000010,0.000010,0.0,0.0,1.0,0.0,0.0,...,0.148112,0.010870,9.996509e-01,0.925926,0.085875,0.148318,0.022161,1.0,1.000000,4


In [53]:
# Extract features and labels
X = data.drop(columns=['Target']).values
y = data['Target'].values
print(f'X: {X.shape}, y: {y.shape}')

X: (96282, 45), y: (96282,)


In [54]:
# Split the data into train, validation, and test sets
# 70% training, 15% validation, 15% testing
X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state=42)
print('Train set:', X_train.shape, y_train.shape)
print('Validation set:', X_val.shape, y_val.shape)
print('Test set:', X_test.shape, y_test.shape)

Train set: (67397, 45) (67397,)
Validation set: (14442, 45) (14442,)
Test set: (14443, 45) (14443,)


In [55]:
# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.int64)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.int64)

# Create data loaders
batch_size = 32
train_loader = DataLoader(TensorDataset(X_train, y_train), 
                          batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size)

In [56]:
# Define the model
class CNN_LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(CNN_LSTM_Model, self).__init__()
        
        # CNN layer for extracting spatial features
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=32, 
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=32, out_channels=64, 
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        # LSTM layer for extracting temporal features
        self.lstm = nn.LSTM(
            input_size=64, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(2)
        x = x.permute(0, 2, 1)
        out = self.cnn(x)
        out = out.permute(0, 2, 1)
        out, _ = self.lstm(out)
        out = self.fc(out[:, -1, :])
        return out

In [57]:
# Initialize the model, loss function, and optimizer
input_size = 1
hidden_size = 128
num_layers = 1
num_classes = 6

model = CNN_LSTM_Model(input_size, hidden_size, num_layers, num_classes)
print(model.to('cpu'))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

CNN_LSTM_Model(
  (cnn): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (lstm): LSTM(64, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=6, bias=True)
)


In [58]:
# Training function
def train(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        
    return running_loss / len(dataloader.dataset)

# Evaluation function
def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Compute additional metrics
    accuracy = correct / len(dataloader.dataset)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    
    return running_loss / len(dataloader.dataset), accuracy, precision, recall, f1

In [60]:
# Train and evaluate the model
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc, val_prec, val_rec, val_f1 = evaluate(model, val_loader, criterion)
    print(f"Epoch {epoch+1}/{num_epochs}: <Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}, Val Acc: {val_acc:.6f}, Val Prec: {val_prec:.6f}, Val Rec: {val_rec:.6f}, Val F1: {val_f1:.6f}>")

Epoch 1/10: <Train Loss: 0.409707, Val Loss: 0.441379, Val Acc: 0.763883, Val Prec: 0.797294, Val Rec: 0.760644, Val F1: 0.749210>
Epoch 2/10: <Train Loss: 0.395351, Val Loss: 0.377525, Val Acc: 0.813391, Val Prec: 0.822776, Val Rec: 0.811980, Val F1: 0.810330>
Epoch 3/10: <Train Loss: 0.382596, Val Loss: 0.376136, Val Acc: 0.812422, Val Prec: 0.824973, Val Rec: 0.810746, Val F1: 0.808783>
Epoch 4/10: <Train Loss: 0.374101, Val Loss: 0.365507, Val Acc: 0.816784, Val Prec: 0.828722, Val Rec: 0.815195, Val F1: 0.814065>
Epoch 5/10: <Train Loss: 0.366084, Val Loss: 0.357705, Val Acc: 0.822047, Val Prec: 0.830503, Val Rec: 0.820733, Val F1: 0.818555>
Epoch 6/10: <Train Loss: 0.361370, Val Loss: 0.359021, Val Acc: 0.812007, Val Prec: 0.816208, Val Rec: 0.811615, Val F1: 0.812327>
Epoch 7/10: <Train Loss: 0.356680, Val Loss: 0.353997, Val Acc: 0.822878, Val Prec: 0.830531, Val Rec: 0.821673, Val F1: 0.822159>
Epoch 8/10: <Train Loss: 0.353168, Val Loss: 0.355097, Val Acc: 0.824401, Val Prec:

In [61]:
# Evaluate on the test set
loss, accuracy, precision, recall, f1 = evaluate(model, test_loader, criterion)
print(f'Test Loss: {loss:.6f}'
      f'\nAccuracy: {accuracy:.6f}'
      f'\nPrecision: {precision:.6f}'
      f'\nRecall: {recall:.6f}'
      f'\nF1-score: {f1:.6f}')

Test Loss: 0.356524
Accuracy: 0.817143
Precision: 0.824726
Recall: 0.816282
F1-score: 0.813118
